In [1]:
CATBOOST_VERSION = 18.0
NFOLDS = 10
SEED = 0
NCOMP = 50
P = 0.03

In [2]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import PowerTransformer 
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder
from sklearn.feature_selection import SelectPercentile, SelectKBest
from sklearn.model_selection import train_test_split, KFold, cross_validate, ShuffleSplit
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import PCA, TruncatedSVD
from sklearn.metrics import mean_squared_error
from sklearn import set_config
from catboost import CatBoostRegressor
import re
import statsmodels.api as sm

In [3]:

X_train = pd.read_csv('data/X_train.csv', encoding='cp949').drop(columns='ID')
y_train = pd.read_csv('data/y_train.csv', encoding='cp949').Salary

X_test = pd.read_csv('data/X_test.csv', encoding='cp949')
test_id = X_test.ID
X_test = X_test.drop(columns='ID')

In [4]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16570 entries, 0 to 16569
Data columns (total 11 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   직종      16570 non-null  object 
 1   세부직종    16570 non-null  object 
 2   직무태그    14055 non-null  object 
 3   근무경력    16570 non-null  object 
 4   근무형태    6661 non-null   object 
 5   근무지역    16570 non-null  object 
 6   출신대학    16570 non-null  object 
 7   대학전공    16570 non-null  object 
 8   어학시험    4988 non-null   object 
 9   자격증     16570 non-null  object 
 10  대학성적    14600 non-null  float64
dtypes: float64(1), object(10)
memory usage: 1.4+ MB


In [5]:
X_train['NaN_count'] = X_train.isnull().sum(1)
X_test['NaN_count'] = X_test.isnull().sum(1)

In [6]:
X_train['NotNull_count'] = X_train.notnull().sum(1) -1
X_test['NotNull_count'] = X_test.notnull().sum(1) -1

In [7]:
X_train.head(2)

,직종,세부직종,직무태그,근무경력,근무형태,근무지역,출신대학,대학전공,어학시험,자격증,대학성적,NaN_count,NotNull_count
0,문화·예술·신문·방송,영상·음향·사진·카메라,"취재기자, 편집 기사, 유아 사이트 기획, 시나리오 작성",0개월,NaN,"서울,경기,서울",성균관대학교,국문,NaN,無,70.0,2,9
1,경영·기획·회계·사무,사무·총무·법무,NaN,2년 11개월,정규직,"부산,서울,일본",신라대학교,관광경영,JLPT,無,NaN,2,9


In [8]:
X_train_af = pd.read_csv('data/X_train_김종윤.csv', encoding='cp949')
y_train_af = pd.read_csv('data/y_train_김종윤.csv', encoding='cp949')

X_test_af = pd.read_csv('data/X_test_김종윤.csv', encoding='cp949')

#### 근무경력-년 월 구분

In [9]:
X_train_a, X_test_a = X_train.copy(), X_test.copy()
X_all = pd.concat([X_train_a,X_test_a], ignore_index=True)

In [10]:
for i in range(len(X_all.근무경력)):
    if len(X_all.근무경력[i]) == 3 or len(X_all.근무경력[i]) == 4:
        X_all.근무경력[i] = '0년 ' + X_all.근무경력[i]

In [11]:
we = X_all.근무경력.str.split(' ',expand=True)
we.columns = ['년','개월']

In [12]:
# year, month로 구분하여 열 생성
we = X_all.근무경력.str.split(' ',expand=True)
we.columns = ['년','개월']

we_y = we.년.str.split('년', expand=True) 
we_y.columns = ['년','삭제']
we_y.drop('삭제',axis=1,inplace=True)
we_y = we_y.astype(int) 

we_m = we.개월.str.split('개월', expand=True)
we_m.columns = ['개월','삭제']
we_m.drop('삭제',axis=1,inplace=True)
we_m = we_m.astype(int)

X_all['근무경력_y'] = we_y['년']
X_all['근무경력_m'] = we_m['개월']

In [13]:
X_train = X_all.iloc[:X_train.shape[0],:].reset_index(drop=True)
X_test = X_all.iloc[X_train.shape[0]:,:].reset_index(drop=True)

In [14]:
### 경력을 수치로 변환

def f1(x):
    numbers = re.findall(r'\d+', x)
    return int(numbers[0])*12 if len(numbers) == 1 else int(numbers[0])*12+int(numbers[1])

X_train['근무경력'] = X_train['근무경력'].apply(f1)
X_test['근무경력'] = X_test['근무경력'].apply(f1)

#### 근무경력 제곱항 추가

In [15]:
X_train['근무경력^2'] = np.square(X_train['근무경력'])
X_test['근무경력^2'] = np.square(X_test['근무경력'])

#### 근무경력 세분화

In [16]:
def seg_worklength(x):
    if 0.0 <= x < 24.0:
        x = '0to2'
    elif 24 <= x < 48:
        x = '2to4'
    elif 48 <= x < 72: 
        x = '4to6'
    elif 72 <= x < 96:
        x = '6to8'
    elif 96 <= x < 120:
        x = '8to10'
    elif 120 <= x:
        x = 'over10'
    return x
    
X_train['근무경력_세분화'] = X_train['근무경력'].apply(seg_worklength)
X_test['근무경력_세분화'] =X_test['근무경력'].apply(seg_worklength)

####  마지막 근무형태

In [17]:
X_train_cp, X_test_cp = X_train.copy(), X_test.copy()

In [18]:
X_train_cp['근무형태'].fillna('N', inplace=True)
X_test_cp['근무형태'].fillna('N', inplace=True)

In [19]:
X_train['마지막근무형태'] = 0
for i,j in enumerate(X_train_cp['근무형태']) :
    X_train_cp['근무형태'][i] = j.rstrip(', ')
for i,j in enumerate(X_train_cp['근무형태'].str.split(', ')):
    X_train['마지막근무형태'][i] = j[-1]



X_test['마지막근무형태'] = 0
for i,j in enumerate(X_test_cp['근무형태']) :
    X_test_cp['근무형태'][i] = j.rstrip(', ')
for i,j in enumerate(X_test_cp['근무형태'].str.split(', ')):
    X_test['마지막근무형태'][i] = j[-1]

#### 해외근무지역 피쳐

In [20]:
korea = ['서울','부산', '경기', '전국', '기타', '전북', '충남', '광주', '인천', '대전', '경남','충북','울산', '경북', '강원','전남'
        ,'대구','제주','광주']
foreign = ['홍콩','미국','해외','중국','캐나다','인도네시아','일본','싱가포르','필리핀','대만','말레이시아','인도네시아','러시아','인도'
          ,'프랑스','필리핀','말레이시아','방글라데시']
world = korea + foreign

f_f = X_train.근무지역.str.split(',',expand=True)[0].isin(foreign)
s_f = X_train.근무지역.str.split(',',expand=True)[1].isin(foreign)
t_f = X_train.근무지역.str.split(',',expand=True)[2].isin(foreign)

c = (f_f | s_f | t_f)

c_index = X_train.loc[c,'근무지역'].index

X_train['해외근무지역'] =  (X_train.index).isin(c_index)

f_f = X_test.근무지역.str.split(',',expand=True)[0].isin(foreign)
s_f = X_test.근무지역.str.split(',',expand=True)[1].isin(foreign)
t_f = X_test.근무지역.str.split(',',expand=True)[2].isin(foreign)

c = (f_f | s_f | t_f)

ct_index = X_test.loc[c,'근무지역'].index

X_test['해외근무지역'] =  (X_test.index).isin(ct_index)

X_train['해외근무지역'] = X_train['해외근무지역'].apply(lambda x: '유' if x== True else '무')
X_test['해외근무지역'] = X_test['해외근무지역'].apply(lambda x: '유' if x== True else '무')

#### 근무지역 abc

In [21]:
a = X_train['근무지역'].str.split(',')
b = []
c = []
d = []
for i in a :
    b.append(i[0])
    c.append(i[1])    
    d.append(i[2])    

b = pd.Series(b)
c = pd.Series(c)
d = pd.Series(d)

a = pd.concat([b,c,d],axis=1)
a.columns = ['근무지역_a','근무지역_b','근무지역_c']

X_train.replace({'근무지역_c':{'':'N'},
                '근무지역_b':{'':'N'}},inplace=True)

X_train = pd.concat([X_train,a], axis=1)

In [22]:
a = X_test['근무지역'].str.split(',')
b = []
c = []
d = []
for i in a :
    b.append(i[0])
    c.append(i[1])    
    d.append(i[2])    

b = pd.Series(b)
c = pd.Series(c)
d = pd.Series(d)

a = pd.concat([b,c,d],axis=1)
a.columns = ['근무지역_a','근무지역_b','근무지역_c']

X_test.replace({'근무지역_c':{'':'N'},
                '근무지역_b':{'':'N'}},inplace=True)

X_test = pd.concat([X_test,a], axis=1)

#### 근무희망형태

In [23]:
X_train['근무희망형태'] = X_train_af['근무희망형태']
X_test['근무희망형태'] = X_test_af['근무희망형태']

X_train['근무희망형태'] = X_train['근무희망형태'].astype(str)
X_test['근무희망형태'] = X_test['근무희망형태'].astype(str)

#### 단과대

In [24]:
df_college = pd.read_csv('data/단과대_최종.csv', encoding='cp949')

In [25]:
인문대 = list(df_college['인문대'])
사과대 = list(df_college['사과대'])
경영경제대 = list(df_college['경영경제대'])
교육대 = list(df_college['교육대'])
예체능 = list(df_college['예체능'])
공대 = list(df_college['공대'])
보건계열 = list(df_college['보건계열'])
자연과학대 = list(df_college['자연과학'])
법대 = list(df_college['법대'])
소프트웨어대 = list(df_college['소프트웨어'])
농축산대 = list(df_college['농축산대'])
건축대 = list(df_college['건축대'])
신학대 = list(df_college['신학대'])
결측 = list(df_college['결측'])

In [26]:
X_train['단과대'] = X_train.대학전공.copy()
X_test['단과대'] = X_test.대학전공.copy()

In [27]:
X_train['단과대'] = X_train['단과대'].replace(인문대, '인문대')
X_train['단과대'] = X_train['단과대'].replace(사과대, '사과대')
X_train['단과대'] = X_train['단과대'].replace(경영경제대, '경영경제대')
X_train['단과대'] = X_train['단과대'].replace(교육대, '교육대')
X_train['단과대'] = X_train['단과대'].replace(예체능, '예체능')
X_train['단과대'] = X_train['단과대'].replace(공대, '공대')
X_train['단과대'] = X_train['단과대'].replace(보건계열, '보건계열')
X_train['단과대'] = X_train['단과대'].replace(자연과학대, '자연과학')
X_train['단과대'] = X_train['단과대'].replace(법대, '법대')
X_train['단과대'] = X_train['단과대'].replace(소프트웨어대, '소프트웨어대')
X_train['단과대'] = X_train['단과대'].replace(농축산대, '농축산대')
X_train['단과대'] = X_train['단과대'].replace(건축대, '건축대')
X_train['단과대'] = X_train['단과대'].replace(신학대, '신학대')
X_train['단과대'] = X_train['단과대'].replace(결측, '결측')

In [28]:
X_test['단과대'] = X_test['단과대'].replace(인문대, '인문대')
X_test['단과대'] = X_test['단과대'].replace(사과대, '사과대')
X_test['단과대'] = X_test['단과대'].replace(경영경제대, '경영경제대')
X_test['단과대'] = X_test['단과대'].replace(교육대, '교육대')
X_test['단과대'] = X_test['단과대'].replace(예체능, '예체능')
X_test['단과대'] = X_test['단과대'].replace(공대, '공대')
X_test['단과대'] = X_test['단과대'].replace(보건계열, '보건계열')
X_test['단과대'] = X_test['단과대'].replace(자연과학대, '자연과학')
X_test['단과대'] = X_test['단과대'].replace(법대, '법대')
X_test['단과대'] = X_test['단과대'].replace(소프트웨어대, '소프트웨어대')
X_test['단과대'] = X_test['단과대'].replace(농축산대, '농축산대')
X_test['단과대'] = X_test['단과대'].replace(건축대, '건축대')
X_test['단과대'] = X_test['단과대'].replace(신학대, '신학대')
X_test['단과대'] = X_test['단과대'].replace(결측, '결측')

In [29]:
X_train['단과대'].value_counts()

경영경제대     4032
공대        3160
인문대       2310
예체능       1879
소프트웨어대    1853
사과대       1245
자연과학      1036
법대         319
교육대        237
건축대        218
농축산대       111
보건계열        81
신학대         61
결측          28
Name: 단과대, dtype: int64

#### id 생성

In [30]:
X_train['ID'] = \
X_train['근무경력'].astype(str) + '_' + X_train['출신대학'].astype(str) + '_' +\
X_train['대학전공'].astype(str) + '_' + X_train['어학시험'].astype(str) + '_' +\
X_train['자격증'].astype(str) + '_' + X_train['대학성적'].astype(str) 

In [31]:
X_test['ID'] = \
X_test['근무경력'].astype(str) + '_' + X_test['출신대학'].astype(str) + '_' +\
X_test['대학전공'].astype(str) + '_' + X_test['어학시험'].astype(str) + '_' +\
X_test['자격증'].astype(str) + '_' + X_test['대학성적'].astype(str) 

#### 대학전공, 경력세분화

In [32]:
X_train['대학전공_and_경력세분화'] = X_train_af['대학전공_and_경력세분화']
X_test['대학전공_and_경력세분화'] = X_test_af['대학전공_and_경력세분화']

#### 어학시험 범주 묶기

In [33]:
X_train.replace({'어학시험':{' ':'N'}}, inplace=True)
X_train.어학시험.fillna('N', inplace=True)

In [34]:
X_train['어학시험언어'] = 'N'

In [35]:
EN = ['TOEIC', 'TOEFL', 'OPIc', 'TEPS', 'IELTS', 'TOEIC Speaking', 'TOEFL-CBT', 'G-TELP', 'TOEFL-IBT', 'TOEFL-PBT', 'SEPT', 'G-ETAT', 'GRE', 'G-TELP(GLT)', 'TOEIC S&W', 'OPI']
JP = ['JLPT', 'JPT', 'JTRA'] 
CH = ['新HSK', 'HSK']
ETC = ['기타', '기타시험']
FR = ['DALF', 'DELF']
for i,j in enumerate(X_train['어학시험']):
    if j in EN:
        X_train['어학시험언어'][i] = 'EN'
    elif j in JP:
        X_train['어학시험언어'][i] = 'JP'
    elif j in CH:
        X_train['어학시험언어'][i] = 'CH'
    elif j in ETC:
        X_train['어학시험언어'][i] = 'ETC'
    elif j in FR:
        X_train['어학시험언어'][i] = 'FR'
        
X_train.head(2)

,직종,세부직종,직무태그,근무경력,근무형태,근무지역,출신대학,대학전공,어학시험,자격증,...,마지막근무형태,해외근무지역,근무지역_a,근무지역_b,근무지역_c,근무희망형태,단과대,ID,대학전공_and_경력세분화,어학시험언어
0,문화·예술·신문·방송,영상·음향·사진·카메라,"취재기자, 편집 기사, 유아 사이트 기획, 시나리오 작성",0,NaN,"서울,경기,서울",성균관대학교,국문,N,無,...,N,무,서울,경기,서울,-1,인문대,0_성균관대학교_국문_nan_無_70.0,세부직종: 동양 근무경력: 0to2,N
1,경영·기획·회계·사무,사무·총무·법무,NaN,35,정규직,"부산,서울,일본",신라대학교,관광경영,JLPT,無,...,정규직,유,부산,서울,일본,1,경영경제대,35_신라대학교_관광경영_JLPT_無_nan,세부직종: 관광 근무경력: 2to4,JP


In [36]:
X_test['어학시험언어'] = 'N'

In [37]:
EN = ['TOEIC', 'TOEFL', 'OPIc', 'TEPS', 'IELTS', 'TOEIC Speaking', 'TOEFL-CBT', 'G-TELP', 'TOEFL-IBT', 'TOEFL-PBT', 'SEPT', 'G-ETAT', 'GRE', 'G-TELP(GLT)', 'TOEIC S&W', 'OPI']
JP = ['JLPT', 'JPT', 'JTRA'] 
CH = ['新HSK', 'HSK']
ETC = ['기타', '기타시험']
FR = ['DALF', 'DELF']
X_test['어학시험언어']='N'
for i,j in enumerate(X_test['어학시험']):
    if j in EN:
        X_test['어학시험언어'][i] = 'EN'
    elif j in JP:
        X_test['어학시험언어'][i] = 'JP'
    elif j in CH:
        X_test['어학시험언어'][i] = 'CH'
    elif j in ETC:
        X_test['어학시험언어'][i] = 'ETC'
    elif j in FR:
        X_test['어학시험언어'][i] = 'FR'
        
X_test.head(2)

,직종,세부직종,직무태그,근무경력,근무형태,근무지역,출신대학,대학전공,어학시험,자격증,...,마지막근무형태,해외근무지역,근무지역_a,근무지역_b,근무지역_c,근무희망형태,단과대,ID,대학전공_and_경력세분화,어학시험언어
0,전문·교육·자격,외국어·번역·통역,"중국어,영어,강사,사무,micro office,호텔서비스",0,NaN,"서울,중국,캐나다",이화여자대학교,증어중문,TOEFL,無,...,N,유,서울,중국,캐나다,-1,인문대,0_이화여자대학교_증어중문_TOEFL_無_80.0,세부직종: 동양 근무경력: 0to2,EN
1,영업·판매·TM,해외영업,해외영업,0,NaN,"서울,,",중앙대학교,경영학과,,有,...,N,무,서울,,,-1,경영경제대,0_중앙대학교_경영학과_ _有_80.0,세부직종: 경영 근무경력: 0to2,N


In [38]:
X_train['어학시험'].fillna('N',inplace = True)
X_train.loc[X_train['어학시험'] == ' ','어학시험'] = 'BLANK'
X_train['상위어학시험'] = '0'
for i in range(16570):
    X_train['상위어학시험'][i] = X_train['어학시험'].loc[i][:5]
X_train.loc[X_train['상위어학시험'] == '新HSK','상위어학시험'] = 'HSK'

list(X_train['상위어학시험'].value_counts().index[12:])

for i in list(X_train['상위어학시험'].value_counts().index[12:]):
    X_train.loc[X_train['상위어학시험'] == i,'상위어학시험'] = '기타'

X_test['어학시험'].fillna('N',inplace = True)
X_test.loc[X_test['어학시험'] == ' ','어학시험'] = 'BLANK'
X_test['상위어학시험'] = '0'
for i in range(11048):
    X_test['상위어학시험'][i] = X_test['어학시험'].loc[i][:5]
X_test.loc[X_train['상위어학시험'] == '新HSK','상위어학시험'] = 'HSK'

for i in list(X_test['상위어학시험'].value_counts().index[12:]):
    X_test.loc[X_test['상위어학시험'] == i,'상위어학시험'] = '기타'
X_train.head(2)

,직종,세부직종,직무태그,근무경력,근무형태,근무지역,출신대학,대학전공,어학시험,자격증,...,해외근무지역,근무지역_a,근무지역_b,근무지역_c,근무희망형태,단과대,ID,대학전공_and_경력세분화,어학시험언어,상위어학시험
0,문화·예술·신문·방송,영상·음향·사진·카메라,"취재기자, 편집 기사, 유아 사이트 기획, 시나리오 작성",0,NaN,"서울,경기,서울",성균관대학교,국문,N,無,...,무,서울,경기,서울,-1,인문대,0_성균관대학교_국문_nan_無_70.0,세부직종: 동양 근무경력: 0to2,N,N
1,경영·기획·회계·사무,사무·총무·법무,NaN,35,정규직,"부산,서울,일본",신라대학교,관광경영,JLPT,無,...,유,부산,서울,일본,1,경영경제대,35_신라대학교_관광경영_JLPT_無_nan,세부직종: 관광 근무경력: 2to4,JP,JLPT


#### 근무지역 - 김종윤

In [39]:
def region(x):
    if x in ['서울']:
        x = '수도권_서울'
    elif x in ['경기']:
        x = '수도권_경기'
    elif x in ['인천']:
        x = '수도권_인천'
    elif x in ['강원']:
        x = '강원'
    elif x in ['경남','경북','대구','부산','울산']:
        x = '영남'
    elif x in ['광주','전남','전북']:
        x = '호남'
    elif x in ['충남','충북']:
        x = '호서'
    elif x in ['제주']:
        x = '제주'
    elif x in ['대만','러시아','말레이시아','방글라데시','싱가포르','인도','인도네시아','일본','중국','필리핀','홍콩']:
        x = '해외_유라시아'
    elif x in ['미국','캐나다']:
        x = '해외_아메리카'
    elif x in ['프랑스','해외']:
        x = '해외_기타'
    elif x == '':
        x = ''
    else: 
        x = '기타'
    return x

def new_region(x):
    x = set(sorted(map(region,x.split(','))))
    x.discard('')
    return tuple(x)


X_train['new_근무지역'] = X_train['근무지역'].apply(new_region)
X_test['new_근무지역'] = X_test['근무지역'].apply(new_region)

#### 근무지역 경험횟수 

In [40]:
X_train['근무지역_경험횟수'] = X_train['new_근무지역'].apply(lambda x: len(x))
X_test['근무지역_경험횟수'] = X_test['new_근무지역'].apply(lambda x: len(x))

#### 근무형태 처리-이준영

data = pd.concat([X_train, X_test]).reset_index()

# 근무 형태 처리
null_list = data[data['근무경력']==0].index
data['근무형태_2'] = data['근무형태']
data['근무형태_2'][null_list] = data['근무형태'][null_list].fillna('경력없음')
data['근무형태_2'] = data['근무형태'].fillna('missing')

#
data['근무형태_2'] = data['근무형태'].str.replace(',', ' ')
data['근무형태_2'] = data['근무형태'].str.strip()

#
hyung_list = []
for i in data['근무형태']:
    if i[:3] == '정규직' and '해외취업' in i:
        i = '정규직(해외o)'
    elif i[:3] == '정규직' and '해외취업' not in i:
        i = '정규직(해외x)'
    hyung_list.append(i)    
data['근무형태_2'] = hyung_list

#
hyun_list2 = []
for i in data['근무형태']:
    if i[:3] == '계약직':
        i = '계약직'
    elif i in ['인턴','파견직']:
        i = '계약직'
    elif i in ['해외취업','병역특례']:
        i = '기타'    
    hyun_list2.append(i)   
data['근무형태_2'] = hyun_list2

### 유형별 피처 분리

In [41]:
X_train.dtypes

직종                 object
세부직종               object
직무태그               object
근무경력                int64
근무형태               object
근무지역               object
출신대학               object
대학전공               object
어학시험               object
자격증                object
대학성적              float64
NaN_count           int64
NotNull_count       int64
근무경력_y              int64
근무경력_m              int64
근무경력^2              int64
근무경력_세분화           object
마지막근무형태            object
해외근무지역             object
근무지역_a             object
근무지역_b             object
근무지역_c             object
근무희망형태             object
단과대                object
ID                 object
대학전공_and_경력세분화     object
어학시험언어             object
상위어학시험             object
new_근무지역           object
근무지역_경험횟수           int64
dtype: object

In [42]:
# X_train.to_csv('data/X_train_yh.csv', encoding='cp949', index=False)
# X_test.to_csv('data/X_test_yh.csv', encoding='cp949', index=False)
# y_train.to_csv('data/y_train_yh.csv', encoding='cp949', index=False)

#### 최종 데이터 불러오기

In [43]:
X_train = pd.read_csv('data/X_train_yh.csv', encoding='cp949')
y_train = pd.read_csv('data/y_train_yh.csv', encoding='cp949')

X_test = pd.read_csv('data/X_test_yh.csv', encoding='cp949')

In [44]:
# 인서울 여부
in_seoul_lst = ['성균관대학교', '중앙대학교', '세종대학교', '연세대학교', '이화여자대학교']

X_train['인서울여부'] = X_train['출신대학'].map(lambda x: '0' if x in in_seoul_lst else '1')

X_test['인서울여부'] = X_test['출신대학'].map(lambda x: '0' if x in in_seoul_lst else '1')

In [45]:
# 근무지역_소분류
X_train['근무지역_소분류'] = X_train['근무지역'].copy()
X_test['근무지역_소분류'] = X_test['근무지역'].copy()

In [46]:
X_train['근무지역_소분류'][X_train['근무지역_소분류'].str.contains
                ('해외|미국|중국|러시아|일본|캐나다|필리핀|인도|싱가포르|대만|말레이시아|프랑스|홍콩')] = '해외'

X_train['근무지역_소분류'][X_train['근무지역_소분류'] == '전국,,'] = '전국'
X_train['근무지역_소분류'][X_train['근무지역_소분류'] == '전국,전국,'] = '전국'
X_train['근무지역_소분류'][X_train['근무지역_소분류'] == '전국,전국,전국'] = '전국'

X_train['근무지역_소분류'][X_train['근무지역_소분류'].str.split(',').str[0] == '서울'] = '서울'
X_train['근무지역_소분류'][X_train['근무지역_소분류'].str.split(',').str[0] == '경기'] = '경기'
X_train['근무지역_소분류'][X_train['근무지역_소분류'].str.split(',').str[0] == '인천'] = '인천'
X_train['근무지역_소분류'][X_train['근무지역_소분류'].str.split(',').str[0] == '부산'] = '부산'
X_train['근무지역_소분류'][X_train['근무지역_소분류'].str.split(',').str[0] == '대전'] = '대전'
X_train['근무지역_소분류'][X_train['근무지역_소분류'].str.split(',').str[0] == '광주'] = '광주'
X_train['근무지역_소분류'][X_train['근무지역_소분류'].str.split(',').str[0] == '충남'] = '충남'
X_train['근무지역_소분류'][X_train['근무지역_소분류'].str.split(',').str[0] == '경남'] = '경남'
X_train['근무지역_소분류'][X_train['근무지역_소분류'].str.split(',').str[0] == '전북'] = '전북'
X_train['근무지역_소분류'][X_train['근무지역_소분류'].str.split(',').str[0] == '충북'] = '충북'
X_train['근무지역_소분류'][X_train['근무지역_소분류'].str.split(',').str[0] == '제주'] = '제주'
X_train['근무지역_소분류'][X_train['근무지역_소분류'].str.split(',').str[0] == '경북'] = '경북'
X_train['근무지역_소분류'][X_train['근무지역_소분류'].str.split(',').str[0] == '강원'] = '강원'
X_train['근무지역_소분류'][X_train['근무지역_소분류'].str.split(',').str[0] == '울산'] = '울산'
X_train['근무지역_소분류'][X_train['근무지역_소분류'].str.split(',').str[0] == '전남'] = '전남'
X_train['근무지역_소분류'][X_train['근무지역_소분류'].str.split(',').str[0] == '대구'] = '대구'
X_train['근무지역_소분류'][X_train['근무지역_소분류'].str.split(',').str[0] == '전북'] = '전북'

X_train['근무지역_소분류'][X_train['근무지역_소분류'].str.contains
                ('기타')] = '기타'

In [47]:
X_test['근무지역_소분류'][X_test['근무지역_소분류'].str.contains
                ('해외|미국|중국|러시아|일본|캐나다|필리핀|인도|싱가포르|대만|말레이시아|프랑스|홍콩')] = '해외'

X_test['근무지역_소분류'][X_test['근무지역_소분류'] == '전국,,'] = '전국'
X_test['근무지역_소분류'][X_test['근무지역_소분류'] == '전국,전국,'] = '전국'
X_test['근무지역_소분류'][X_test['근무지역_소분류'] == '전국,전국,전국'] = '전국'

X_test['근무지역_소분류'][X_test['근무지역_소분류'].str.split(',').str[0] == '서울'] = '서울'
X_test['근무지역_소분류'][X_test['근무지역_소분류'].str.split(',').str[0] == '경기'] = '경기'
X_test['근무지역_소분류'][X_test['근무지역_소분류'].str.split(',').str[0] == '인천'] = '인천'
X_test['근무지역_소분류'][X_test['근무지역_소분류'].str.split(',').str[0] == '부산'] = '부산'
X_test['근무지역_소분류'][X_test['근무지역_소분류'].str.split(',').str[0] == '대전'] = '대전'
X_test['근무지역_소분류'][X_test['근무지역_소분류'].str.split(',').str[0] == '광주'] = '광주'
X_test['근무지역_소분류'][X_test['근무지역_소분류'].str.split(',').str[0] == '충남'] = '충남'
X_test['근무지역_소분류'][X_test['근무지역_소분류'].str.split(',').str[0] == '경남'] = '경남'
X_test['근무지역_소분류'][X_test['근무지역_소분류'].str.split(',').str[0] == '전북'] = '전북'
X_test['근무지역_소분류'][X_test['근무지역_소분류'].str.split(',').str[0] == '충북'] = '충북'
X_test['근무지역_소분류'][X_test['근무지역_소분류'].str.split(',').str[0] == '제주'] = '제주'
X_test['근무지역_소분류'][X_test['근무지역_소분류'].str.split(',').str[0] == '경북'] = '경북'
X_test['근무지역_소분류'][X_test['근무지역_소분류'].str.split(',').str[0] == '강원'] = '강원'
X_test['근무지역_소분류'][X_test['근무지역_소분류'].str.split(',').str[0] == '울산'] = '울산'
X_test['근무지역_소분류'][X_test['근무지역_소분류'].str.split(',').str[0] == '전남'] = '전남'
X_test['근무지역_소분류'][X_test['근무지역_소분류'].str.split(',').str[0] == '대구'] = '대구'
X_test['근무지역_소분류'][X_test['근무지역_소분류'].str.split(',').str[0] == '전북'] = '전북'

X_test['근무지역_소분류'][X_test['근무지역_소분류'].str.contains
                ('기타')] = '기타'

In [48]:
# 8.1 출신대학 + 근무경력 조합

# sum

group_object1 = X_train.groupby(by=['출신대학'])['근무경력'].agg('sum').reset_index()
group_object2 = X_test.groupby(by=['출신대학'])['근무경력'].agg('sum').reset_index()

group_object1 = group_object1.rename(columns = {'근무경력':'대학_경력_sum'})
group_object2 = group_object2.rename(columns = {'근무경력':'대학_경력_sum'})

X_train = pd.merge(X_train, group_object1, on = '출신대학', how='left')
X_test = pd.merge(X_test, group_object2, on = '출신대학', how='left')

In [49]:
# 8.1 출신대학 + 근무경력 조합

# mean

group_object3 = X_train.groupby(by=['출신대학'])['근무경력'].agg('mean').reset_index()
group_object4 = X_test.groupby(by=['출신대학'])['근무경력'].agg('mean').reset_index()

group_object3 = group_object3.rename(columns = {'근무경력':'대학_경력_mean'})
group_object4 = group_object4.rename(columns = {'근무경력':'대학_경력_mean'})

X_train = pd.merge(X_train, group_object3, on = '출신대학', how='left')
X_test = pd.merge(X_test, group_object4, on = '출신대학', how='left')

In [50]:
# variance

group_object5 = X_train.groupby(by=['출신대학'])['근무경력'].agg('var').reset_index()
group_object6 = X_test.groupby(by=['출신대학'])['근무경력'].agg('var').reset_index()

group_object5 = group_object5.rename(columns = {'근무경력':'대학_경력_variance'})
group_object6 = group_object6.rename(columns = {'근무경력':'대학_경력_variance'})

X_train = pd.merge(X_train, group_object5, on = '출신대학', how='left')
X_test = pd.merge(X_test, group_object6, on = '출신대학', how='left')

In [51]:
# standard deviation

group_object7 = X_train.groupby(by=['출신대학'])['근무경력'].agg('std').reset_index()
group_object8 = X_test.groupby(by=['출신대학'])['근무경력'].agg('std').reset_index()

group_object7 = group_object7.rename(columns = {'근무경력':'대학_경력_standard deviation'})
group_object8 = group_object8.rename(columns = {'근무경력':'대학_경력_standard deviation'})

X_train = pd.merge(X_train, group_object7, on = '출신대학', how='left')
X_test = pd.merge(X_test, group_object8, on = '출신대학', how='left')

In [52]:
# 변수형 피처 추가
X_train['근무경력_x_대학성적'] = X_train['근무경력'] * X_train['대학성적']
X_test['근무경력_x_대학성적'] = X_test['근무경력'] * X_test['대학성적']

In [53]:
# 인서울대학 피처 추가
in_seoul = ['성균관대학교', '중앙대학교', '연세대학교', '한성대학교', '성신여자대학교', '동덕여자대학교', '이화여자대학교',
            '서울여자대학교', '서울과학기술대학교', '세종대학교']

not_in_seoul = ['경기대학교', '동아대학교', '수원대학교', '동의대학교', '전주대학교', '상지대학교', '한밭대학교', '한림대학교', '인천대학교', '선문대학교', '군산대학교', '세명대학교',
                '광주대학교', '동서대학교', '호남대학교', '서원대학교', '용인대학교', '호원대학교', '목포대학교', '신라대학교', '성결대학교', '제주대학교', '남부대학교',
                '한국산업기술대학교', '협성대학교', '경주대학교', '한세대학교', '성공회대학교', '서울신학대학교', '한일장신대학교', '송원대학교', '부산디지털대학교']

In [54]:
# X_train['서울4년제대학'] = X_train.출신대학.copy()
# X_test['서울4년제대학'] = X_test.출신대학.copy()

In [55]:
# X_train['서울4년제대학'] = X_train['서울4년제대학'].replace(in_seoul, 'yes')
# X_train['서울4년제대학'] = X_train['서울4년제대학'].replace(not_in_seoul, 'no')

# X_test['서울4년제대학'] = X_test['서울4년제대학'].replace(in_seoul, 'yes')
# X_test['서울4년제대학'] = X_test['서울4년제대학'].replace(not_in_seoul, "no")

In [56]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16570 entries, 0 to 16569
Data columns (total 37 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   직종                        16570 non-null  object 
 1   세부직종                      16570 non-null  object 
 2   직무태그                      14055 non-null  object 
 3   근무경력                      16570 non-null  int64  
 4   근무형태                      6661 non-null   object 
 5   근무지역                      16570 non-null  object 
 6   출신대학                      16570 non-null  object 
 7   대학전공                      16570 non-null  object 
 8   어학시험                      16570 non-null  object 
 9   자격증                       16570 non-null  object 
 10  대학성적                      14600 non-null  float64
 11  NaN_count                 16570 non-null  int64  
 12  NotNull_count             16570 non-null  int64  
 13  근무경력_y                    16570 non-null  int64  
 14  근무경력_m

In [57]:
# numeric_features = ['근무경력','대학성적', '근무경력_x_대학성적', '대학_경력_standard deviation', '대학_경력_variance', '대학_경력_mean','대학_경력_sum']
# categorical_features = ['직종','세부직종','출신대학','대학전공','어학시험','자격증', '인서울여부', '근무형태_계정(해)','근무지역_소분류','해외근무지역', 'ID']
# binary_features = ['직무태그','근무지역','근무형태'] 

In [58]:
numeric_features = ['근무경력','대학성적','근무경력^2', '근무경력_x_대학성적','대학_경력_standard deviation', '대학_경력_variance', '대학_경력_mean','대학_경력_sum']
categorical_features = ['직종','세부직종','출신대학','대학전공','어학시험','자격증','근무경력_세분화','마지막근무형태','ID',
                       '근무지역_a','근무지역_b','근무지역_c','해외근무지역','단과대','대학전공_and_경력세분화',
                       '근무경력_y', '근무경력_m','근무희망형태', '근무지역_소분류'] # '인서울여부'
binary_features = ['직무태그','근무지역','근무형태']

In [59]:
X_train = X_train[numeric_features+categorical_features+binary_features]  # 순서 주의!!!
X_test = X_test[numeric_features+categorical_features+binary_features]

# CatBoost의 cat_features 파라미터에 지정할 범주형 피처 위치
cat_index = [list(X_train.columns).index(c) for c in categorical_features]

### 전처리 파이프라인 구축

In [60]:
# 상하한값 제한을 통한 결측값 처리 함수: FunctionTransformer를 통해 호출
def remove_outlier(X, q=0.05):  
    df = pd.DataFrame(X)
    return df.apply(lambda x: x.clip(x.quantile(q), x.quantile(1-q)), axis=0).values

# 회귀분석의 계수검정을 이용한 피처선택 전처리기 클래스
class MyFeatureSelector(TransformerMixin, BaseEstimator):
    # 전처리기 생성 즉, MyFeatureSelector() 호출시 실행
    def __init__(self, p=0.01):
        self.p = p

    # 전처리기의 fit() 호출시 실행
    def fit(self, X, y=None):
        X = sm.add_constant(X)
        results = sm.OLS(y, X).fit()
        self.cols = list(results.pvalues[1:] <= self.p)
        return self
    
    # 전처리기의 transform() 호출시 실행
    def transform(self, X):
        return X[:,self.cols].astype(np.int64)        
    
numeric_transformer = Pipeline(
    steps=[
        ("imputer", SimpleImputer(strategy="mean")),
        ("outlier", FunctionTransformer(remove_outlier, kw_args={'q':0.05})),
    ]
)

categorical_transformer = Pipeline(
    steps=[
        ("imputer", SimpleImputer(strategy="most_frequent")), 
        ("encoder", OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=99999, dtype=np.object)),
    ]
)

binary_transformer = Pipeline(
    steps=[
        ("impuer", FunctionTransformer(lambda x: x.fillna('없음'))),      
        ("corpus", FunctionTransformer(lambda x: x.str.replace('·',',').str.split(',').str.join(" "))),
        ("BoW", CountVectorizer()),
        ("dense", FunctionTransformer(lambda x: x.toarray().astype(int), accept_sparse=True)),
    ]
)

column_transformer = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features),
        ("bin1", make_pipeline(binary_transformer, TruncatedSVD(n_components=NCOMP,random_state=SEED)), binary_features[0]),
        ("bin2", make_pipeline(binary_transformer, MyFeatureSelector(p=P)), binary_features[1]),
        ("bin3", make_pipeline(binary_transformer, MyFeatureSelector(p=P)), binary_features[2]),
    ]
)

preprocessor = Pipeline(
    steps=[
        ("column", column_transformer), 
    ]
)

set_config(display="diagram")
preprocessor

Pipeline(steps=[('column',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('outlier',
                                                                   FunctionTransformer(func=<function remove_outlier at 0x7fc8a7a6a4c0>,
                                                                                       kw_args={'q': 0.05}))]),
                                                  ['근무경력', '대학성적', '근무경력^2',
                                                   '근무경력_x_대학성적',
                                                   '대학_경력_standard deviation',
                                                   '대학_경력_variance',
                                                   '대학_경력_mean', '대학_경력_sum']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer'...
                                                  Pipeline(steps=[('pipeline',
                                                                   Pipeline(steps=[('impuer',
                                                                                    FunctionTransformer(func=<function <lambda> at 0x7fc8a7a6a8b0>)),
                                                                                   ('corpus',
                                                                                    FunctionTransformer(func=<function <lambda> at 0x7fc8a7a6a820>)),
                                                                                   ('BoW',
                                                                                    CountVectorizer()),
                                                                                   ('dense',
                                                                                    FunctionTransformer(accept_sparse=True,
                                                                                                        func=<function <lambda> at 0x7fc8a7a6a790>))])),
                                                                  ('myfeatureselector',
                                                                   MyFeatureSelector(p=0.03))]),
                                                  '근무형태')]))])

### 데이터 전처리

In [61]:
X_train = preprocessor.fit_transform(X_train, y_train)
X_test = preprocessor.transform(X_test)

### 모형생성

In [62]:
%%time

# 최적화된 하이퍼파라미터로 OOF를 수행하여 최종 CatBoost 모형 생성:
# No tuning => tuning한 모델에 비해 성능이 떨어지지 않음

#sscv = ShuffleSplit(test_size=.3334, n_splits=5, random_state=0)
models = cross_validate(CatBoostRegressor(cat_features=cat_index, verbose=False, random_state=SEED),
                        X_train, y_train, 
                        cv=NFOLDS, 
                        scoring='neg_mean_squared_error', 
                        return_estimator=True)
oof_pred = np.array([m.predict(X_test) for m in models['estimator']]).mean(axis=0)

scores = models['test_score']
print("\nCatBoost CV scores: ", np.sqrt(-1*scores))
print("CatBoost CV mean = %.2f" % np.sqrt(-1*scores.mean()), "with std = %.2f" % np.sqrt(scores.std()))


CatBoost CV scores:  [733.73835886 766.28883574 741.73722848 746.11383395 827.60444185
 821.77815453 816.26990527 918.02158626 890.20005916 990.98718292]
CatBoost CV mean = 829.23 with std = 371.64
CPU times: user 32min 23s, sys: 1min 48s, total: 34min 12s
Wall time: 3min 42s


In [63]:
# # submission 화일 생성
# filename = f'catboost_{CATBOOST_VERSION}_{np.sqrt(-1*scores.mean()):.2f}.csv'
# pd.DataFrame({'ID':test_id, 'Salary':oof_pred}).to_csv(filename, index=False)

- 이렇게 나와야됨.

1) CatBoost CV mean = 829.27 with std = 371.59 
-> Public score : 799.3711

2) 인서울여부 피처 추가(채원) -> 범주형 변수 추가

-> CatBoost CV mean = 828.98 with std = 372.72

3) 근무지역_소분류 피처 추가(채원) -> 범주형 변수 추가

-> CatBoost CV mean = 828.12 with std = 367.30

4) '대학_경력_standard deviation', '대학_경력_variance', '대학_경력_mean','대학_경력_sum' -> 수치형변수 추가

-> CatBoost CV mean = 828.02 with std = 371.92

5) '근무경력_x_대학성적' -> 수치형 변수 추가

-> CatBoost CV mean = 828.18 with std = 373.77

6) '서울4년제대학' -> 범주형 변수 추가

-> CatBoost CV mean = 828.39 with std = 370.11

7) 인서울여부 / '서울4년제대학' -> 피처 2개 추가

-> CatBoost CV mean = 830.28 with std = 369.80

8) 근무지역_소분류 / '대학_경력_standard deviation', '대학_경력_variance', '대학_경력_mean','대학_경력_sum'

-> CatBoost CV mean = 828.75 with std = 369.82

9) 근무지역_소분류 /  '대학_경력_mean','대학_경력_sum'

-> CatBoost CV mean = 828.97 with std = 374.40

10) 근무지역_소분류 / '근무경력_x_대학성적'

-> CatBoost CV mean = 830.67 with std = 375.53

11) 모두 - 서울4년제 대학 제외 

-> CatBoost CV mean = 827.77 with std = 367.87

12) 모두

-> CatBoost CV mean = 828.88 with std = 372.65

13) 모두 - 서울4년제 대학 제외 - 인서울여부

-> CatBoost CV mean = 829.23 with std = 371.64

